In [1]:
import datetime
import os
import time
from tqdm import tqdm
import torch
import h5py
import numpy as np
from torchinfo import summary
import math
from torch.autograd import Variable
import pickle

C:\Users\yuins\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\yuins\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\yuins\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
# custom module
import model, dataloader, train_model, test_model
import importlib
importlib.reload(model)
importlib.reload(dataloader)
importlib.reload(train_model)
importlib.reload(test_model)
from model import *
from dataloader import *
from train_model import train
from test_model import test

# Parameters

In [3]:
config = {
    "root" : "./data/pickles/",
    "train_feature" : "AtrialFibrilation_feature.pickle",
    "train_target" : "AtrialFibrilation_target.pickle",
    "test_feature" : "AtrialFibrilation_feature_test.pickle",
    "test_target" : "AtrialFibrilation_target_test.pickle",
    "labelencodedict" : { 'n' : 0,
                    's' : 1,
                    't' :2,
                   },
    "epochs" : 3000,
    "learning_rate" : 1e-4,
    # model 
    "first_out_channels" : 32,
    "first_kernel_size" : 5,
    "second_in_channels" : 32,
    "second_out_channels" : 16,
    "second_kernel_size" : 3,
    "dim_model" : 64,
    "dim_inner" : 128,
    "num_heads" : 8,
    "dropout_rate" : 0.15,
    "squeeze_factor" : 2,
    "sec_kernel_size" : 3,
    "sec_stride" : 1,
    "N" : 3,
    "gamma" : 2,
    "u" : 2,        
    "weight_decay" : 0.1,
       "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Main

In [4]:
if __name__ == "__main__":
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]
    weight_decay = config["weight_decay"]

    X_train, y_train, train_y, X_test, y_test, test_y = load_dataset(config)
    
    config["num_samples"] = X_train.shape[0]
    config["first_in_channels"] = X_train.shape[1]
    config["L"] = X_train.shape[2]
    config["length"] = X_train.shape[2]
    
    md = Net(
                dim_model = config["dim_model"],
                gamma = config["gamma"],
                u = config["u"],
                ys = y_train,
                first_in_channels = config["first_in_channels"],
                first_out_channels = config["first_out_channels"],
                first_kernel_size = config["first_kernel_size"],
                second_in_channels = config["second_in_channels"],
                second_out_channels = config["second_out_channels"],
                second_kernel_size = config["second_kernel_size"],
                N = config["N"],  #slice
                L = config["L"], # slice
                dim_inner = config["dim_inner"],
                num_samples = config["num_samples"], 
                length = config["length"], 
                num_heads = config["num_heads"],
                dropout_rate = config["dropout_rate"],
                squeeze_factor = config["squeeze_factor"],
                sec_kernel_size = config["sec_kernel_size"],
                sec_stride = config["sec_stride"],
                num_classes = len(config["labelencodedict"].keys())).to(config['device'])
    optimizer = torch.optim.Adam(md.parameters(), weight_decay=weight_decay, lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    train_loss, train_acc = train(X_train, train_y, optimizer, model = md, epochs = epochs)
    md.ys = y_test
    test_loss, test_acc = test(X_test, test_y, model = md)

C:\Users\yuins\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\yuins\Hanyang_23_1st\AI\model.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)
C:\Users\yuins\Hanyang_23_1st\AI\model.py:254: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)


Epoch: 0, Loss: 0.6918, Train acc: 91.1111%
Epoch: 1, Loss: 0.6733, Train acc: 86.6667%
Epoch: 2, Loss: 0.7120, Train acc: 95.5556%
Epoch: 3, Loss: 0.5623, Train acc: 86.6667%
Epoch: 4, Loss: 0.6719, Train acc: 95.5556%
Epoch: 5, Loss: 0.7093, Train acc: 82.2222%
Epoch: 6, Loss: 0.6832, Train acc: 91.1111%
Epoch: 7, Loss: 0.6309, Train acc: 95.5556%
Epoch: 8, Loss: 0.4707, Train acc: 91.1111%
Epoch: 9, Loss: 0.6828, Train acc: 91.1111%
Epoch: 10, Loss: 0.5566, Train acc: 91.1111%
Epoch: 11, Loss: 0.5926, Train acc: 95.5556%
Epoch: 12, Loss: 0.4866, Train acc: 95.5556%
Epoch: 13, Loss: 0.5845, Train acc: 100.0000%
Epoch: 14, Loss: 0.4175, Train acc: 100.0000%
Epoch: 15, Loss: 0.4061, Train acc: 100.0000%
Epoch: 16, Loss: 0.4052, Train acc: 100.0000%
Epoch: 17, Loss: 0.3485, Train acc: 100.0000%
Epoch: 18, Loss: 0.3155, Train acc: 100.0000%
Epoch: 19, Loss: 0.2555, Train acc: 100.0000%
Epoch: 20, Loss: 0.3211, Train acc: 100.0000%
Epoch: 21, Loss: 0.3144, Train acc: 100.0000%
Epoch: 22, 

KeyboardInterrupt: 

In [ ]:
md.ys = y_test
test_loss, test_acc = test(X_test, test_y, model = md)